In [ ]:
import numpy as np
import organs.system as system
from organs.index import Index

In [ ]:
p = system.init()

n = len(Index)
y0 = np.zeros(n)
y0[Index.plasma_glucose] = 5.5 #gmmol
y0[Index.plasma_insulin] = 1.2 #g/L
y0[Index.plasma_fattyacid] = 0.4 #mmol/L
y0[Index.plasma_aminoacid] = 0.2 #mmol/L
y0[Index.vsc_TAG] = 10 * 1000 /p.V.vsc *.1 # NOTE: NEED UNITS

# volume of skeletal muscle is ~25.0L
y0[Index.muscle_NAD] = 1.6*(60*.4)/25 #mmol/L 
y0[Index.muscle_NADH] = 0.15*(60*.4)/25 #mmol/L

#no certain data for FAD and FADH2, but assume its around 1/3 of NAD
y0[Index.muscle_FAD] = 1.6*(60*.4)/25/3 # NOTE: NEED UNITS
y0[Index.muscle_FADH2] = 0.15*(60*.4)/25/3 # NOTE: NEED UNITS

y0[Index.gut_glucose] = 50 # NOTE: NEED UNITS
y0[Index.gut_fructose] = 30 # NOTE: NEED UNITS
y0[Index.plasma_fructose] = 3 # NOTE: NEED UNITS
y0[Index.micellar_fattyacid] = 100 # NOTE: NEED UNITS